# Runner for Experiment 1


## Experiment

In [1]:
from qiskit import assemble
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.passes import DenseLayout, ExVF2Layout, \
                                     NoiseAdaptiveLayout, SabreLayout

from exp.custom_passmanager import custom_pass_manager

In [6]:
def exp1_evaluate(circuit, layout_method, backend, ideal=True, seed=None, layout_reps=100):
 
    coupling_map = CouplingMap(backend.configuration().coupling_map)
    backend_properties = backend.properties()

    if layout_method == 'exvf2':
        layout = ExVF2Layout(coupling_map)
    elif layout_method == 'dense':
        layout = DenseLayout(coupling_map, None if ideal else backend_properties)
    elif layout_method == 'noise_adaptive':
        layout = NoiseAdaptiveLayout(backend_properties)
    elif layout_method == 'sabre':
        layout = SabreLayout(coupling_map, max_iterations=1, seed=seed)
    else:
        raise Exception('layout_method unknown %s' % layout_method)

    passmanager = custom_pass_manager(backend, layout, layout_reps=layout_reps)

    times = {}
    count_ops_after_map = {}

    def callback(**kwargs):
        times[kwargs['pass_'].name()] = times.get(kwargs['pass_'].name(), 0) + kwargs['time']
        if kwargs['pass_'].name() == 'StochasticSwap':
            count_ops_after_map.update(kwargs['dag'].count_ops())
    
    transpiled = passmanager.run(circuit, callback=callback)

    needed_swaps = 0 if passmanager.property_set['is_swap_mapped'] else count_ops_after_map['swap']
    time = times[layout.name()] / layout_reps

    return time, needed_swaps

/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Runner

In [7]:
import pandas as pd
from qiskit.test.mock.backends import FakeManhattan

from exp.circuit_generators import graphstate_manhatten_ring
from exp.write_to_df import write_to_dataframe

In [8]:
def exp1_run(layout_method, exp_path, num_samples=100, num_qubits=12, num_shots=8192):
    backend = FakeManhattan()
    circuit = graphstate_manhatten_ring(num_qubits)

    df = pd.DataFrame(columns=["seed", "time", "swaps_needed"])
    for num_index in range(num_samples):
        seed = num_index
        time, swaps_needed = exp1_evaluate(
            circuit, layout_method, backend=backend, ideal=True, seed=seed)
        total_result = {'seed': seed,
                        'time': time,
                        'swaps_needed': swaps_needed}

        df = df.append(total_result, ignore_index=True)

    print(df)

## Execution

In [9]:
layout_methods = [
    'exvf2',
    'dense',
    'noise_adaptive',
    'sabre',
]
exp_path = "./data/"
num_samples = 100
for method in layout_methods:
    exp1_run(method, exp_path, num_samples=num_samples)

    seed      time  swaps_needed
0    0.0  0.000004           0.0
1    1.0  0.000004           0.0
2    2.0  0.000003           0.0
3    3.0  0.000003           0.0
4    4.0  0.000003           0.0
..   ...       ...           ...
95  95.0  0.000002           0.0
96  96.0  0.000002           0.0
97  97.0  0.000002           0.0
98  98.0  0.000002           0.0
99  99.0  0.000003           0.0

[100 rows x 3 columns]
    seed      time  swaps_needed
0    0.0  0.000156           7.0
1    1.0  0.000151           7.0
2    2.0  0.000161           7.0
3    3.0  0.000144           7.0
4    4.0  0.000141           7.0
..   ...       ...           ...
95  95.0  0.000169           7.0
96  96.0  0.000172           7.0
97  97.0  0.000174           7.0
98  98.0  0.000182           7.0
99  99.0  0.000182           7.0

[100 rows x 3 columns]
    seed      time  swaps_needed
0    0.0  0.000161          11.0
1    1.0  0.000163          11.0
2    2.0  0.000159          11.0
3    3.0  0.000181          